# Format data
Dans ce fichier, on importe les differentes sources, et on les normalise et formatte pour utiliser un meme format

## Intro: installation

In [11]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [12]:
import pandas as pd
import re

## Fichier corpus

In [ ]:
# Inspire par: http://abdulbaqi.io/2019/01/19/quranic-roots-pandas/

# Lire le fichier de quranic corpus (en sautant les lignes d'intro)
corpus = pd.read_csv('./original_data/quranic-corpus-morphology-0.4.txt', sep='\t',skiprows=56)



# L'enregistrer converti en csv
corpus.to_csv('./original_data/quranic-corpus-morphology.csv')



In [ ]:
# Voir un sample 
corpus.head()

In [ ]:
# Compter lignes
corpus.count()

In [ ]:
# Ajouter une colonne pour "root"


# Function to extract the value of 'root' from 'features' using regex
def extraire_root(features):
    match = re.search(r"\|ROOT:([^|]+)", features)
    if match:
        return match.group(1)
    else:
        return None

# Apply the 'extract_root' function to create the 'root' column
corpus['root'] = corpus['FEATURES'].apply(extraire_root)


corpus.head()

In [ ]:
# Fonction pour extraire la position du mot au niveau 3 (par exemple 6:11:2)
def extraire_position_mot(location):
    match = re.match(r"\((\d+:\d+:\d+)", location)
    if match:
        return match.group(1)
    else:
        return None

# Appliquer la fonction pour creer "word_location"
corpus['word_location'] = corpus['LOCATION'].apply(extraire_position_mot)

corpus.head()

In [ ]:
# Extraire le verset de 'LOCATION'
def extraire_verset(location):
    match = re.match(r"\((\d+:\d++)", location)
    if match:
        return match.group(1)
    else:
        return None

# Apply the 'extraire_verset' function to create the 'verse' column
corpus['verse'] = corpus['LOCATION'].apply(extraire_verset)


corpus.head()

In [ ]:
# compter le nombre de mots selon cette source
corpus['word_location'].nunique()

## Fichier Coran mot a mot

In [ ]:
coran_mot = pd.read_csv('./data/Coran_mot_a_mot_coranseul_12juin2019.csv')
coran_mot.head()

In [ ]:
# compter le nombre de mots selon cette source
coran_mot['N° de mot'].nunique()

In [ ]:
# Il y a une disparite. On va maintenant trouver ou. 
# Je vais creer la colonne word_location sur coran_mot aussi avec le meme format que dans l'autre.
# Formatter 'N° de mot' en format 1:1:1
def formatter_numero_en_word_location(numero):
    return numero.replace("#", ":")

coran_mot['word_location'] = coran_mot['N° de mot'].apply(formatter_numero_en_word_location)

coran_mot.head()

In [ ]:
# Trouver les word_location qui sont dans une source et pas l'autre


# Un set recupere les valeurs uniques
corpus_word_locations = set(corpus['word_location'])
coran_mot_word_locations = set(coran_mot['word_location'])

# les word_location dans corpus qui ne sont pas dans coran_mot
seulement_dans_corpus = corpus_word_locations - coran_mot_word_locations

# Find the word_location values that appear in df2 but not in df1
seulement_dans_coran_mot = coran_mot_word_locations - corpus_word_locations

# Print the results
print("Dans corpus mais pas coran_mot:", seulement_dans_corpus)
print("Dans coran_mot mais pas corpus:", seulement_dans_coran_mot)

In [ ]:
# En gros, il manque les versets 9:128 et 9:129 dans Coran mot a mot. Il manque aussi le mot 96:5:5.
# Hypothese: il y a un decalage? Comparons le verset precedent, 9:127.

# d'abord je cree une versino de corpus qui aggregue les FORM au niveau de word_location. 

corpus_word = corpus.groupby('word_location')['FORM'].apply(lambda x: ' '.join(map(str, x))).reset_index(name='transliteration_corpus')

corpus_word.head()

In [ ]:
# Ensuite je compare les deux au verset 9:127
joined = corpus_word.merge(coran_mot, on="word_location", how="outer")

joined_9_127 = joined.query('word_location.str.startswith("9:127")')
joined_9_127.head()
# Ca m'a l'air correct. Donc ca veut dire qu'il manque 9:128 et 9:129 dans coran mot a mot. 
# Je ne comprends pas trop pourquoi ces deux versets manquent.

In [ ]:

# Par contre pourquoi le mot 96:5:5, c'est surement plus anodin. Regardons:

joined_95_5 = joined.query('word_location.str.startswith("96:5")')
joined_95_5.head()

# Effectivement cette difference m'a l'air anodine! 
# Corpus considere que lamo est un mot separe et coran mot a mot non.